In [1]:
NU_COPULA = 6.
N_SIM = 100000

In [2]:
import pandas as pd
import numpy as np

In [3]:
import os

dir_ = './transformed_input'

udls = pd.read_csv(os.path.join(dir_, 'UDL.csv'), header=0, index_col=0)
positions = pd.read_csv(os.path.join(dir_, 'positions.csv'), header=0, index_col=0)

In [4]:
udl_labels = udls.index.tolist()

## Random variables

In [5]:
correl_matrix = pd.read_excel('./input_files/correlation_matrix.xlsx')
correl_matrix.set_index('Pearson', inplace=True)

correl_matrix = correl_matrix[udl_labels]
correl_matrix = correl_matrix.transpose()[udl_labels]

covar_matrix = correl_matrix

In [6]:
from scipy.stats import multivariate_normal, chi2, t

gauss_rv = multivariate_normal(cov=covar_matrix).rvs(N_SIM).transpose()
chi2_rv = chi2(NU_COPULA).rvs(N_SIM)

mult_factor = np.sqrt(NU_COPULA / chi2_rv)

student_rvs = np.multiply(mult_factor, gauss_rv)
inv_student_rvs = t.cdf(student_rvs, NU_COPULA)

print inv_student_rvs.shape

gauss_rv = None
chi2_rv = None
mult_factor = None
student_rvs = None

(90L, 100000L)


In [7]:
losses_rvs = np.ndarray(inv_student_rvs.shape)
losses_rvs[:] = np.nan

udls_trans = udls.transpose()

for i, col in enumerate(udls_trans.columns):
    params = udls_trans[col]
    
    nu = params['Nu']
    coeff = params['Coef']
    val = params['UDL value']
    
    crt_ppf = t.ppf(inv_student_rvs[i, :], nu)
    
    losses_rvs[i, :] = -val * coeff * crt_ppf
    
losses_rvs

array([[ -2.00918163e-01,  -1.98904574e-01,   8.05547682e-01, ...,
          7.02859260e-03,  -3.34334806e-01,   3.81628349e-01],
       [ -1.15058457e+00,  -3.67421826e+00,   1.77541888e+00, ...,
          7.59314048e+00,  -2.28884150e-02,   1.00063290e+01],
       [ -5.86133209e-02,  -2.14075038e-01,   8.95697494e-01, ...,
          2.77796415e-01,   5.93673260e-01,   5.19736725e-01],
       ..., 
       [  1.81851590e-02,  -7.23759219e-01,   1.95555926e+00, ...,
         -1.62176401e-01,   5.58272297e-01,   7.09532721e-02],
       [ -7.78091804e-02,  -9.95199374e-01,   5.52746588e-01, ...,
          3.99998478e-01,   9.13688940e-02,   1.36533745e+00],
       [ -2.33881625e-01,  -6.04025869e-02,  -6.25818832e-04, ...,
          4.72676402e-01,  -1.15245243e+00,   7.15670886e-01]])

In [8]:
losses_df = pd.DataFrame(losses_rvs, index=udl_labels, columns=[r'$\omega_{%i}$' % i for i in xrange(N_SIM)])
losses_df

,$\omega_{0}$,$\omega_{1}$,$\omega_{2}$,$\omega_{3}$,$\omega_{4}$,$\omega_{5}$,$\omega_{6}$,$\omega_{7}$,$\omega_{8}$,$\omega_{9}$,...,$\omega_{99990}$,$\omega_{99991}$,$\omega_{99992}$,$\omega_{99993}$,$\omega_{99994}$,$\omega_{99995}$,$\omega_{99996}$,$\omega_{99997}$,$\omega_{99998}$,$\omega_{99999}$
AAI,-0.200918,-0.198905,0.805548,0.583668,-1.706628,-0.103197,0.201652,0.456547,0.640914,-0.505325,...,-0.078274,0.433510,-0.610485,-2.820888,1.985151,0.685405,0.003559,0.007029,-0.334335,0.381628
AEX,-1.150585,-3.674218,1.775419,3.664648,-6.985859,-2.927309,4.360093,0.959643,7.777305,-1.862348,...,0.578672,9.481559,-2.389425,-19.523459,15.682777,12.078140,-0.842923,7.593140,-0.022888,10.006329
AF,-0.058613,-0.214075,0.895697,0.269507,-0.315589,0.223574,0.277623,0.176089,0.149691,0.026785,...,0.000244,0.476596,-0.171294,-0.314697,0.383086,0.578728,-0.336255,0.277796,0.593673,0.519737
AGN,-0.026015,-0.138477,-0.099280,0.048946,-0.389255,0.095240,0.171380,0.146761,0.049076,-0.077695,...,0.019834,0.033442,-0.003632,-0.563368,0.931487,0.290534,0.027892,0.029634,0.158926,0.065737
AH,0.159044,-0.289840,1.701097,0.155359,-2.697122,-0.132729,0.242296,0.048544,0.368971,0.045669,...,0.140038,-0.144903,0.378085,0.039735,0.190374,0.162137,0.286393,0.991165,0.129587,0.751473
AHA,-0.073544,0.706702,0.143321,0.661208,0.674901,-1.047396,0.783650,0.249500,0.708006,-0.217687,...,-0.292071,2.313801,-1.198159,-0.472021,2.477632,1.177141,-0.368198,0.040826,1.827546,1.475319
AI,0.198540,-0.159675,-0.008743,-0.022520,-0.532079,-5.583746,0.675201,0.607766,-0.486662,-2.262635,...,0.777259,1.880116,0.539625,-3.751192,2.118027,1.207875,0.435211,2.028794,-3.808177,1.480489
AKZ,-0.774371,-0.257924,-0.697137,0.594904,-1.951710,-0.784442,0.962759,-1.325928,1.529619,-0.612038,...,1.241070,1.447305,-0.509786,0.819816,2.309963,1.017815,-0.886466,0.972120,-0.470946,2.013650
AL1,-0.108167,-0.366591,0.114330,-0.274369,0.498826,0.482861,-0.389156,0.090225,0.872098,0.215726,...,0.136355,0.006785,-0.848368,-0.841086,1.008975,0.436896,0.213393,0.242586,-0.064690,0.922087
ASL,-0.254408,1.470753,7.680641,2.308929,-0.608044,1.496898,1.358193,0.904747,1.379871,-2.390325,...,2.264694,4.211668,-1.458178,-11.921133,3.663944,1.775062,-0.844696,3.703363,-0.357974,2.882644


In [9]:
import os

dir_ = './simulations'

if not os.path.exists(dir_):
    os.makedirs(dir_)

In [10]:
losses_df.to_csv(os.path.join(dir_, 'nu_eq_%i.csv' % NU_COPULA))